In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score
import pickle

In [4]:
df = pd.read_csv("IMDB-Dataset.csv")
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [5]:
df.columns

Index(['review', 'sentiment'], dtype='object')

In [6]:
df["review"].isnull().unique()

array([False])

In [7]:
df["sentiment"].isnull().unique()

array([False])

In [8]:
df["sentiment"].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

In [9]:
df["sentiment"].replace("negative", 1, inplace=True)


In [10]:
df["sentiment"].replace("positive", 0, inplace=True)

In [11]:
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,0
1,A wonderful little production. <br /><br />The...,0
2,I thought this was a wonderful way to spend ti...,0
3,Basically there's a family where a little boy ...,1
4,"Petter Mattei's ""Love in the Time of Money"" is...",0
5,"Probably my all-time favorite movie, a story o...",0
6,I sure would like to see a resurrection of a u...,0
7,"This show was an amazing, fresh & innovative i...",1
8,Encouraged by the positive comments about this...,1
9,If you like original gut wrenching laughter yo...,0


In [12]:
df["review"][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [13]:
stWords = stopwords.words("english")
ss = SnowballStemmer("english")
def TextPreprocessing(text):
    rem = ""
    cleaned = re.compile(r"<.*?>")
    cleaned = re.sub(cleaned, "", text)
    cleaned = cleaned.lower()
    text = " ".join([word for word in word_tokenize(cleaned) if word not in stWords])
    for t in text:
        if t.isalnum():
            rem = rem + t
        else:
            rem = rem + " "
    text = "".join([ss.stem(t) for t in rem])
    return text

In [14]:
df["review"] = df.review.apply(TextPreprocessing)

In [15]:
df["review"][1]

'wonderful little production   filming technique unassuming  old time bbc fashion gives comforting   sometimes discomforting   sense realism entire piece   actors extremely well chosen  michael sheen    got polari    voices pat   truly see seamless editing guided references williams   diary entries   well worth watching terrificly written performed piece   masterful production one great master  s comedy life   realism really comes home little things   fantasy guard   rather use traditional  dream   techniques remains solid disappears   plays knowledge senses   particularly scenes concerning orton halliwell sets   particularly flat halliwell  s murals decorating every surface   terribly well done  '

In [16]:
X = np.array(df.iloc[:, 0].values)

In [17]:
y = np.array(df["sentiment"].values)

In [18]:
cv = CountVectorizer(max_features=1000)

In [19]:
X = cv.fit_transform(X).toarray()

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=9)

In [21]:
print(X_train.shape, X_test.shape)

(40000, 1000) (10000, 1000)


In [22]:
print(y_train.shape, y_test.shape)

(40000,) (10000,)


In [23]:
gnb, mnb, bnb = GaussianNB(), MultinomialNB(alpha= 1.0, fit_prior=True), BernoulliNB(alpha=0.1, fit_prior=True)

In [24]:
gnb.fit(X_train, y_train)
gnb.predict(X_test)

GaussianNB()

In [30]:
mnb.fit(X_train, y_train)
mnb.predict(X_test)

array([1, 0, 0, ..., 0, 0, 1], dtype=int64)

In [32]:
bnb.fit(X_train, y_train)
bnb.predict(X_test)

array([1, 0, 0, ..., 0, 0, 1], dtype=int64)

In [26]:
y_test

array([1, 0, 0, ..., 1, 0, 1], dtype=int64)

In [27]:
accuracy_score(gnb.predict(X_test), y_test)

0.8141

In [35]:
mnb = accuracy_score(mnb.predict(X_test), y_test)

In [33]:
accuracy_score(bnb.predict(X_test), y_test)

0.8326

In [36]:
pickle.dump(mnb, open("model1.pkl", "wb"))

In [34]:
rev =  """Terrible. Complete trash. Brainless tripe. Insulting to anyone who isn't an 8 year old fan boy. Im actually pretty disgusted that this movie is making the money it is - what does it say about the people who brainlessly hand over the hard earned cash to be 'entertained' in this fashion and then come here to leave a positive 8.8 review?? Oh yes, they are morons. Its the only sensible conclusion to draw. How anyone can rate this movie amongst the pantheon of great titles is beyond me.

So trying to find something constructive to say about this title is hard...I enjoyed Iron Man? Tony Stark is an inspirational character in his own movies but here he is a pale shadow of that...About the only 'hook' this movie had into me was wondering when and if Iron Man would knock Captain America out...Oh how I wished he had :( What were these other characters anyways? Useless, bickering idiots who really couldn't organise happy times in a brewery. The film was a chaotic mish mash of action elements and failed 'set pieces'...

I found the villain to be quite amusing.

And now I give up. This movie is not robbing any more of my time but I felt I ought to contribute to restoring the obvious fake rating and reviews this movie has been getting on IMDb."""

In [38]:
bow,words = [],word_tokenize(TextPreprocessing(rev))

In [42]:
for word in words:
    bow.append(words.count(word))


In [45]:
word_dict = cv.vocabulary_
pickle.dump(word_dict,open('bow.pkl','wb'))

In [48]:
inp = []
for i in word_dict:
    inp.append(TextPreprocessing(rev).count(i[0]))
y_pred = bnb.predict(np.array(inp).reshape(1,1000))

In [49]:
y_pred

array([1], dtype=int64)